In [3]:
import rasterio
from rasterio.warp import calculate_default_transform, reproject, Resampling

def convert_to_wgs84(input_tiff, output_tiff):
    with rasterio.open(input_tiff) as src:
        # Define the target coordinate system (WGS 1984)
        dst_crs = 'EPSG:4326'

        # Calculate the transformation parameters
        transform, width, height = calculate_default_transform(src.crs, dst_crs, src.width, src.height, *src.bounds)

        # Update metadata for the new dataset
        kwargs = src.meta.copy()
        kwargs.update({
            'crs': dst_crs,
            'transform': transform,
            'width': width,
            'height': height
        })

        # Create the output raster
        with rasterio.open(output_tiff, 'w', **kwargs) as dst:
            # Perform the reprojection
            for i in range(1, src.count + 1):
                reproject(
                    source=rasterio.band(src, i),
                    destination=rasterio.band(dst, i),
                    src_transform=src.transform,
                    src_crs=src.crs,
                    dst_transform=transform,
                    dst_crs=dst_crs,
                    resampling=Resampling.nearest
                )

# Example usage
input_tiff = "./EMAG2_V3_UpCont_DataTiff.tif"
output_tiff = "output_in_wgs84.tif"

convert_to_wgs84(input_tiff, output_tiff)
print("Conversion to WGS 1984 complete. Output saved as:", output_tiff)


Conversion to WGS 1984 complete. Output saved as: output_in_wgs84.tif
